

### **📋 Estructura del Análisis**
1. **Carga y Configuración Inicial**
2. **Filtrado Geográfico (Antioquia)**  
3. **Evaluación de Calidad de Datos**
4. **Limpieza de Precios Inválidos**
5. **Validación y Corrección Geográfica**
6. **Filtrado por Tipo de Propiedad**
7. **Análisis de Valores Faltantes**
8. **Text Mining de Descripciones**
9. **Estrategia de Integración de Datos**
10. **Análisis de Variables de Superficie**
11. **Tratamiento Final de Valores Faltantes**
12. **Validación y Resumen Final**

---

*Desarrollado como parte del análisis de mercado inmobiliario de Antioquia*

## 1️⃣ **Carga y Configuración Inicial**

### **🎯 Objetivo de Esta Sección**
Establecer el entorno de trabajo y cargar el dataset inicial de propiedades inmobiliarias colombianas.

### **📚 Justificación de Librerías**
- `pandas`: Manipulación eficiente de datasets grandes
- `numpy`: Operaciones numéricas optimizadas  
- `matplotlib/seaborn`: Visualización de patrones de datos
- `re`: Text mining de descripciones (procesamiento de lenguaje natural)
- `sqlalchemy`: Conexión a base de datos PostgreSQL (opcional)

In [1]:
# ===============================================================
# IMPORTACIÓN DE LIBRERÍAS ESENCIALES
# ===============================================================

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import warnings

# Configuración para optimizar visualización
plt.style.use('default')
sns.set_palette("husl")
warnings.filterwarnings('ignore')

# Configuración de pandas para mejor visualización
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth', 100)

print("✅ Librerías importadas exitosamente")
print("✅ Configuración de visualización establecida")

✅ Librerías importadas exitosamente
✅ Configuración de visualización establecida


In [2]:
# ===============================================================
# CARGA DEL DATASET INICIAL
# ===============================================================

# Ruta al archivo CSV con datos de propiedades colombianas
csv_path = "../data/co_properties.csv"

# Cargar dataset completo
df_original = pd.read_csv(csv_path)

# Información básica del dataset
print("🏠 DATASET CARGADO: Propiedades Inmobiliarias Colombia")
print("=" * 55)
print(f"📊 Dimensiones: {df_original.shape}")
print(f"📋 Registros: {df_original.shape[0]:,}")
print(f"📋 Variables: {df_original.shape[1]}")
print(f"💾 Tamaño en memoria: {df_original.memory_usage(deep=True).sum() / 1024**2:.1f} MB")

# Vista previa de primeras filas
print(f"\n📋 PRIMERAS 3 FILAS:")
df_original.head(3)

🏠 DATASET CARGADO: Propiedades Inmobiliarias Colombia
📊 Dimensiones: (1000000, 25)
📋 Registros: 1,000,000
📋 Variables: 25
💾 Tamaño en memoria: 1482.7 MB

📋 PRIMERAS 3 FILAS:


,id,ad_type,start_date,end_date,created_on,lat,lon,l1,l2,l3,l4,l5,l6,rooms,bedrooms,bathrooms,surface_total,surface_covered,price,currency,price_period,title,description,property_type,operation_type
0,KsjahK62rxcYKXXQjOdkqw==,Propiedad,2020-10-07,2021-10-09,2020-10-07,3.9210,-76.506000,Colombia,Valle del Cauca,NaN,NaN,NaN,NaN,NaN,6.0,7.0,NaN,NaN,1.300000e+09,COP,NaN,Casa Campestre en venta en darien 3469064,"HERMOSA CASA CAMPESTRE, &Aacute;REA 6,000 MT, UBICADA EN LA VIA BUGA - BUENAVENTURA, EN PARCELAC...",Casa,Venta
1,Y+gsBZYq1zu5NoR3V5oUGA==,Propiedad,2020-10-07,2021-01-06,2020-10-07,3.3577,-76.541811,Colombia,Valle del Cauca,Cali,Ciudad Jardín,NaN,NaN,NaN,NaN,7.0,NaN,NaN,2.800000e+09,COP,NaN,Casa en ciudsd jardin,Casa independiente con posiciona en ciudad jardín y hermosos jardines. La casa s eve de amoblada...,Casa,Venta
2,Jpzqxj8/Vgf3Aa5ASxUBNg==,Propiedad,2020-10-07,2020-10-07,2020-10-07,3.3577,-76.541811,Colombia,Valle del Cauca,Cali,Ciudad Jardín,NaN,NaN,NaN,NaN,7.0,NaN,NaN,2.800000e+09,COP,Mensual,Casa en ciudsd jardin,Casa independiente con posiciona en ciudad jardín y hermosos jardines. Amo la casa al\nSur se la...,Casa,Venta


## 2️⃣ **Filtrado Geográfico: Enfoque en Antioquia**



In [4]:
# ===============================================================
# FILTRADO GEOGRÁFICO: ANTIOQUIA CON ANÁLISIS COMPLETO
# ===============================================================


# 2. Filtrar únicamente Antioquia  
df_antioquia = df_original[df_original['l2'] == 'Antioquia'].copy()
df_antioquia = df_antioquia.drop(columns=['id'])  # Eliminar columna 'id'

df_antioquia.to_csv('../data/properties_antioquia.csv', index=False)

# 3. Resumen del filtrado geográfico
print(f"\n✅ RESULTADO DEL FILTRADO GEOGRÁFICO:")
print("=" * 45)
print(f"📊 Dataset original: {len(df_original):,} propiedades")
print(f"📊 Dataset Antioquia: {len(df_antioquia):,} propiedades") 
print(f"📈 Porcentaje conservado: {(len(df_antioquia)/len(df_original)*100):.1f}%")
print(f"📍 Enfoque regional: Departamento de Antioquia únicamente")




✅ RESULTADO DEL FILTRADO GEOGRÁFICO:
📊 Dataset original: 1,000,000 propiedades
📊 Dataset Antioquia: 341,453 propiedades
📈 Porcentaje conservado: 34.1%
📍 Enfoque regional: Departamento de Antioquia únicamente


## 3️⃣ **Evaluación de Calidad de Datos**

### **🎯 Objetivo**
Identificar problemas de calidad que afecten el modelo de predicción de precios.

### **🔍 Estrategia**
1. **Valores faltantes** por variable
2. **Registros duplicados** completos
3. **Rangos lógicos** en variables numéricas
4. **Consistencia** en variables categóricas

In [5]:
# ===============================================================
# EVALUACIÓN RÁPIDA DE CALIDAD DE DATOS
# ===============================================================

print("🔍 EVALUACIÓN DE CALIDAD - DATASET ANTIOQUIA")
print("=" * 50)

# 1. Información básica
print(f"📊 Dimensiones: {df_antioquia.shape}")
print(f"📊 Tipos de datos:")
print(df_antioquia.dtypes.value_counts())

# 2. Valores faltantes críticos
print(f"\n🚨 VALORES FALTANTES (Top 10):")
missing = df_antioquia.isnull().sum()
missing_pct = (missing / len(df_antioquia)) * 100
missing_top = missing[missing > 0].sort_values(ascending=False).head(10)

for col, count in missing_top.items():
    pct = missing_pct[col]
    print(f"   {col:<20}: {count:>8,} ({pct:>5.1f}%)")

# 3. Duplicados
duplicados = df_antioquia.duplicated().sum()
print(f"\n📋 REGISTROS DUPLICADOS: {duplicados:,}")

# 4. Problemas en variables clave
print(f"\n⚠️ PROBLEMAS DETECTADOS:")

# Precios inválidos
precios_invalidos = (df_antioquia['price'] <= 0).sum()
print(f"   • Precios ≤ 0: {precios_invalidos:,}")



🔍 EVALUACIÓN DE CALIDAD - DATASET ANTIOQUIA
📊 Dimensiones: (341453, 24)
📊 Tipos de datos:
object     16
float64     8
Name: count, dtype: int64

🚨 VALORES FALTANTES (Top 10):
   l6                  :  341,453 (100.0%)
   l5                  :  341,453 (100.0%)
   surface_total       :  334,859 ( 98.1%)
   surface_covered     :  334,412 ( 97.9%)
   price_period        :  314,524 ( 92.1%)
   rooms               :  305,597 ( 89.5%)
   l4                  :  262,728 ( 76.9%)
   bedrooms            :  261,106 ( 76.5%)
   lat                 :  196,750 ( 57.6%)
   lon                 :  196,750 ( 57.6%)

📋 REGISTROS DUPLICADOS: 924

⚠️ PROBLEMAS DETECTADOS:
   • Precios ≤ 0: 7


## **Eliminamos la columna l5 y l6**


In [6]:
df_antioquia.drop(['l5', 'l6'], axis=1, inplace=True)

In [7]:
# ===============================================================
# ELIMINAR PRECIOS INVÁLIDOS Y FILTRAR POR MONEDA
# ===============================================================

print("💰 ANÁLISIS DE MONEDA Y LIMPIEZA DE PRECIOS")
print("=" * 50)

inicial = len(df_antioquia)
print(f"📊 Dataset inicial: {inicial:,} registros")

# PASO 1: Verificar existencia y contenido de la columna currency
print(f"\n🔍 PASO 1: ANÁLISIS DE LA COLUMNA 'currency'")
print("-" * 45)

if 'currency' in df_antioquia.columns:
    print("✅ Columna 'currency' encontrada")
    
    # Verificar valores faltantes
    currency_nulos = df_antioquia['currency'].isna().sum()
    currency_completitud = ((len(df_antioquia) - currency_nulos) / len(df_antioquia)) * 100
    
    print(f"📊 Datos faltantes: {currency_nulos:,} ({100-currency_completitud:.1f}%)")
    print(f"📊 Datos completos: {len(df_antioquia) - currency_nulos:,} ({currency_completitud:.1f}%)")
    
    # Mostrar distribución de monedas
    print(f"\n📋 DISTRIBUCIÓN DE MONEDAS:")
    currency_dist = df_antioquia['currency'].value_counts(dropna=False)
    for moneda, count in currency_dist.items():
        porcentaje = (count / len(df_antioquia)) * 100
        estado = "✅" if str(moneda) == "COP" else "❌" if pd.notna(moneda) else "⚠️"
        print(f"   {estado} {moneda}: {count:,} ({porcentaje:.1f}%)")
    
    # PASO 2: Filtrar solo COP
    print(f"\n🎯 PASO 2: FILTRO DE MONEDA - SOLO PESOS COLOMBIANOS")
    print("-" * 55)
    
    antes_currency = len(df_antioquia)
    
    # Filtrar solo registros con currency = 'COP'
    df_cop = df_antioquia[df_antioquia['currency'] == 'COP'].copy()
    
    despues_currency = len(df_cop)
    eliminados_currency = antes_currency - despues_currency
    
    print(f"📊 Antes del filtro: {antes_currency:,}")
    print(f"✅ Después del filtro: {despues_currency:,}")
    print(f"🗑️ Eliminados (otras monedas): {eliminados_currency:,}")
    print(f"📈 Conservado: {(despues_currency/antes_currency*100):.1f}%")
    
    # Usar el dataset filtrado por moneda
    df_para_precio = df_cop.copy()
    
else:
    print("❌ Columna 'currency' NO encontrada")
    print("   → Asumiendo que todos los precios están en pesos colombianos")
    df_para_precio = df_antioquia.copy()

# PASO 3: Eliminar precios inválidos (≤ 0)
print(f"\n🏷️ PASO 3: ELIMINAR PRECIOS INVÁLIDOS")
print("-" * 40)

antes_precio = len(df_para_precio)
print(f"📊 Antes: {antes_precio:,}")

# Eliminar precios ≤ 0
df_clean = df_para_precio[df_para_precio['price'] > 0].copy()

despues_precio = len(df_clean)
eliminados_precio = antes_precio - despues_precio

print(f"✅ Después: {despues_precio:,}")
print(f"🗑️ Eliminados (precios ≤ 0): {eliminados_precio:,}")
print(f"📈 Conservado: {(despues_precio/antes_precio*100):.1f}%")

# RESUMEN FINAL
print(f"\n✅ RESUMEN FINAL DE LIMPIEZA:")
print("=" * 35)
print(f"📊 Dataset inicial: {inicial:,}")
print(f"📊 Dataset final: {len(df_clean):,}")
eliminados_total = inicial - len(df_clean)
print(f"🗑️ Total eliminados: {eliminados_total:,}")
print(f"📈 Conservación total: {(len(df_clean)/inicial*100):.1f}%")
print(f"💰 Moneda confirmada: Pesos Colombianos (COP)")

💰 ANÁLISIS DE MONEDA Y LIMPIEZA DE PRECIOS
📊 Dataset inicial: 341,453 registros

🔍 PASO 1: ANÁLISIS DE LA COLUMNA 'currency'
---------------------------------------------
✅ Columna 'currency' encontrada
📊 Datos faltantes: 79 (0.0%)
📊 Datos completos: 341,374 (100.0%)

📋 DISTRIBUCIÓN DE MONEDAS:
   ✅ COP: 341,366 (100.0%)
   ⚠️ nan: 79 (0.0%)
   ❌ USD: 7 (0.0%)
   ❌ ARS: 1 (0.0%)

🎯 PASO 2: FILTRO DE MONEDA - SOLO PESOS COLOMBIANOS
-------------------------------------------------------
📊 Antes del filtro: 341,453
✅ Después del filtro: 341,366
🗑️ Eliminados (otras monedas): 87
📈 Conservado: 100.0%

🏷️ PASO 3: ELIMINAR PRECIOS INVÁLIDOS
----------------------------------------
📊 Antes: 341,366
✅ Después: 341,365
🗑️ Eliminados (precios ≤ 0): 1
📈 Conservado: 100.0%

✅ RESUMEN FINAL DE LIMPIEZA:
📊 Dataset inicial: 341,453
📊 Dataset final: 341,365
🗑️ Total eliminados: 88
📈 Conservación total: 100.0%
💰 Moneda confirmada: Pesos Colombianos (COP)


## 5️⃣ **Eliminar Coordenadas Inválidas**

### **🎯 Decisión**
Eliminar registros con coordenadas fuera de Antioquia.

### **✅ Justificación**
Variables geográficas son críticas para predicción de precios.

In [8]:
# ===============================================================
# ELIMINAR COORDENADAS FUERA DE ANTIOQUIA
# ===============================================================

print("🌍 VALIDACIÓN GEOGRÁFICA: ANTIOQUIA")
print("=" * 40)

# Límites geográficos de Antioquia (aproximados)
# Basado en los extremos del departamento
LAT_MIN, LAT_MAX = 5.4, 8.8    # Norte-Sur de Antioquia
LON_MIN, LON_MAX = -77.2, -73.8  # Oeste-Este de Antioquia

inicial = len(df_clean)
print(f"📊 Antes: {inicial:,} registros")

# Verificar si hay coordenadas disponibles
coords_disponibles = (df_clean['lat'].notna() & df_clean['lon'].notna()).sum()
print(f"📍 Registros con coordenadas: {coords_disponibles:,}")

if coords_disponibles > 0:
    # PASO 1: Mostrar rango actual de coordenadas
    print(f"\n📊 RANGO ACTUAL DE COORDENADAS:")
    print("-" * 35)
    lat_actual_min = df_clean['lat'].min()
    lat_actual_max = df_clean['lat'].max()
    lon_actual_min = df_clean['lon'].min()
    lon_actual_max = df_clean['lon'].max()
    
    print(f"Latitud:  {lat_actual_min:.3f} a {lat_actual_max:.3f}")
    print(f"Longitud: {lon_actual_min:.3f} a {lon_actual_max:.3f}")
    
    # PASO 2: Identificar coordenadas fuera de Antioquia
    print(f"\n🎯 LÍMITES VÁLIDOS PARA ANTIOQUIA:")
    print("-" * 35)
    print(f"Latitud:  {LAT_MIN} a {LAT_MAX}")
    print(f"Longitud: {LON_MIN} a {LON_MAX}")
    
    # Crear máscara para coordenadas inválidas
    coords_invalidas = (
        (df_clean['lat'].notna()) & (df_clean['lon'].notna()) &
        ((df_clean['lat'] < LAT_MIN) | (df_clean['lat'] > LAT_MAX) |
         (df_clean['lon'] < LON_MIN) | (df_clean['lon'] > LON_MAX))
    )
    
    invalidas_count = coords_invalidas.sum()
    print(f"\n🚨 Coordenadas fuera de Antioquia: {invalidas_count:,}")
    
    # PASO 3: Mostrar ejemplos de coordenadas inválidas (si las hay)
    if invalidas_count > 0:
        print(f"\n📋 EJEMPLOS DE COORDENADAS INVÁLIDAS:")
        ejemplos_invalidos = df_clean[coords_invalidas][['lat', 'lon', 'l3']].head()
        print(ejemplos_invalidos)
        
        # Eliminar registros con coordenadas inválidas
        print(f"\n🗑️ ELIMINANDO COORDENADAS FUERA DE ANTIOQUIA...")
        df_clean = df_clean[~coords_invalidas].copy()
    else:
        print("✅ Todas las coordenadas están dentro de los límites de Antioquia")

else:
    print("⚠️ No hay coordenadas disponibles para validar")

# RESULTADO FINAL
final = len(df_clean)
eliminados = inicial - final

print(f"\n✅ RESULTADO DE LA VALIDACIÓN:")
print("-" * 35)
print(f"Antes:     {inicial:,} registros")
print(f"Después:   {final:,} registros")
print(f"Eliminados: {eliminados:,} registros")
print(f"Conservado: {(final/inicial*100):.1f}%")

if eliminados > 0:
    print(f"💡 Razón: Coordenadas fuera de los límites de Antioquia")
else:
    print(f"💡 Todas las coordenadas están geo-validadas para Antioquia")

🌍 VALIDACIÓN GEOGRÁFICA: ANTIOQUIA
📊 Antes: 341,365 registros
📍 Registros con coordenadas: 144,647

📊 RANGO ACTUAL DE COORDENADAS:
-----------------------------------
Latitud:  -75.640 a 51.801
Longitud: -97.494 a 100.477

🎯 LÍMITES VÁLIDOS PARA ANTIOQUIA:
-----------------------------------
Latitud:  5.4 a 8.8
Longitud: -77.2 a -73.8

🚨 Coordenadas fuera de Antioquia: 291

📋 EJEMPLOS DE COORDENADAS INVÁLIDAS:
             lat        lon           l3
13653   4.710989 -74.072092     Medellín
18685  10.942133 -74.797872     Medellín
18693   4.535000 -75.675689     Medellín
18718   5.060380 -75.489099  La Estrella
20022   5.051645 -75.481864     Medellín

🗑️ ELIMINANDO COORDENADAS FUERA DE ANTIOQUIA...

✅ RESULTADO DE LA VALIDACIÓN:
-----------------------------------
Antes:     341,365 registros
Después:   341,074 registros
Eliminados: 291 registros
Conservado: 99.9%
💡 Razón: Coordenadas fuera de los límites de Antioquia


## 6️⃣ **Filtros por Tipo de Propiedad**

**Decisión:** Enfocarse en apartamentos y casas únicamente

**Justificación:** Los lotes, locales comerciales y fincas tienen dinámicas de precio completamente diferentes. Concentrarse en vivienda residencial garantiza homogeneidad en el análisis

In [9]:
# ===============================================================
# FILTRAR TIPOS DE PROPIEDAD RESIDENCIAL
# ===============================================================

print("🏠 TIPOS DE PROPIEDAD")
print("-" * 25)

inicial = len(df_clean)
print(f"📊 Antes: {inicial:,}")

# Identificar tipos únicos
tipos_unicos = df_clean['property_type'].value_counts()
print(f"\n📝 Tipos de propiedad:")
for tipo, count in tipos_unicos.head(10).items():
    print(f"   {tipo}: {count:,}")

# Filtrar solo vivienda residencial
tipos_residenciales = ['Casa', 'Apartamento', 'casa', 'apartamento']
df_clean = df_clean[df_clean['property_type'].isin(tipos_residenciales)].copy()

final = len(df_clean)
eliminados = inicial - final

print(f"\n✅ Después filtro: {final:,}")
print(f"🗑️ Eliminados: {eliminados:,}")
print(f"📈 Conservado: {(final/inicial*100):.1f}%")

🏠 TIPOS DE PROPIEDAD
-------------------------
📊 Antes: 341,074

📝 Tipos de propiedad:
   Apartamento: 236,178
   Casa: 41,438
   Otro: 38,434
   Lote: 15,344
   Local comercial: 4,681
   Oficina: 3,623
   Finca: 1,140
   Depósito: 211
   Parqueadero: 25

✅ Después filtro: 277,616
🗑️ Eliminados: 63,458
📈 Conservado: 81.4%


In [10]:
# ===============================================================
# ANÁLISIS COMPARATIVO: ROOMS vs BEDROOMS
# ===============================================================

print("🏠 ANÁLISIS COMPARATIVO: ROOMS vs BEDROOMS")
print("=" * 50)

# Ver muestra de ambas columnas
print("📋 MUESTRA DE DATOS (primeras 10 filas):")
print(df_clean[['rooms', 'bedrooms']].head(10))

# PASO 1: Análisis de valores faltantes
print(f"\n📊 ANÁLISIS DE VALORES FALTANTES:")
print("-" * 40)
rooms_nulos = df_clean['rooms'].isna().sum()
bedrooms_nulos = df_clean['bedrooms'].isna().sum()
total_registros = len(df_clean)

print(f"Rooms faltantes:    {rooms_nulos:,} ({rooms_nulos/total_registros*100:.1f}%)")
print(f"Bedrooms faltantes: {bedrooms_nulos:,} ({bedrooms_nulos/total_registros*100:.1f}%)")

# PASO 2: Comparación de valores donde ambos existen
print(f"\n🔍 COMPARACIÓN DONDE AMBAS COLUMNAS TIENEN DATOS:")
print("-" * 50)

# Filtrar registros donde ambas columnas tienen valores
ambas_completas = df_clean[df_clean['rooms'].notna() & df_clean['bedrooms'].notna()].copy()
registros_comparables = len(ambas_completas)

print(f"Registros comparables: {registros_comparables:,}")

if registros_comparables > 0:
    # Verificar si son exactamente iguales
    valores_identicos = (ambas_completas['rooms'] == ambas_completas['bedrooms']).sum()
    valores_diferentes = registros_comparables - valores_identicos
    
    print(f"\n✅ Valores idénticos: {valores_identicos:,} ({valores_identicos/registros_comparables*100:.1f}%)")
    print(f"❌ Valores diferentes: {valores_diferentes:,} ({valores_diferentes/registros_comparables*100:.1f}%)")
    
    # PASO 3: Análisis de diferencias
    if valores_diferentes > 0:
        print(f"\n📊 ANÁLISIS DE DIFERENCIAS:")
        print("-" * 30)
        
        # Crear columna de diferencias
        ambas_completas['diferencia'] = ambas_completas['rooms'] - ambas_completas['bedrooms']
        
        # Estadísticas de diferencias
        diff_stats = ambas_completas['diferencia'].describe()
        print(f"Diferencia promedio: {diff_stats['mean']:.2f}")
        print(f"Diferencia mínima: {diff_stats['min']:.0f}")
        print(f"Diferencia máxima: {diff_stats['max']:.0f}")
        
        # Distribución de diferencias
        print(f"\n📋 DISTRIBUCIÓN DE DIFERENCIAS (rooms - bedrooms):")
        diff_dist = ambas_completas['diferencia'].value_counts().sort_index()
        for diff_val, count in diff_dist.head(10).items():
            pct = (count / registros_comparables) * 100
            print(f"   Diferencia {diff_val:>3.0f}: {count:>6,} casos ({pct:>5.1f}%)")
        
        # Mostrar algunos ejemplos de diferencias
        print(f"\n🔍 EJEMPLOS DE REGISTROS CON DIFERENCIAS:")
        diferentes = ambas_completas[ambas_completas['diferencia'] != 0][['rooms', 'bedrooms', 'diferencia']].head()
        print(diferentes)

# PASO 4: Recomendación
print(f"\n💡 CONCLUSIÓN Y RECOMENDACIÓN:")
print("-" * 35)

if registros_comparables > 0:
    porcentaje_identicos = (valores_identicos / registros_comparables) * 100
    
    if porcentaje_identicos > 90:
        print("✅ Las columnas son MUY SIMILARES (>90% idénticas)")
        print("   → Recomendación: Usar una sola columna (bedrooms es más estándar)")
    elif porcentaje_identicos > 70:
        print("⚠️ Las columnas son PARCIALMENTE SIMILARES (70-90% idénticas)")
        print("   → Recomendación: Revisar diferencias y decidir cuál usar")
    else:
        print("❌ Las columnas son DIFERENTES (<70% idénticas)")
        print("   → Recomendación: Mantener ambas o investigar más")
else:
    print("⚠️ No hay suficientes datos para comparar")
    print("   → Recomendación: Usar la columna con más datos disponibles")

🏠 ANÁLISIS COMPARATIVO: ROOMS vs BEDROOMS
📋 MUESTRA DE DATOS (primeras 10 filas):
     rooms  bedrooms
46     NaN       NaN
47     NaN      19.0
48     NaN       NaN
69     4.0       4.0
74     4.0       4.0
92     3.0       3.0
93     4.0       4.0
104    3.0       3.0
157    5.0       5.0
158    7.0       7.0

📊 ANÁLISIS DE VALORES FALTANTES:
----------------------------------------
Rooms faltantes:    244,778 (88.2%)
Bedrooms faltantes: 206,500 (74.4%)

🔍 COMPARACIÓN DONDE AMBAS COLUMNAS TIENEN DATOS:
--------------------------------------------------
Registros comparables: 32,838

✅ Valores idénticos: 32,837 (100.0%)
❌ Valores diferentes: 1 (0.0%)

📊 ANÁLISIS DE DIFERENCIAS:
------------------------------
Diferencia promedio: -0.00
Diferencia mínima: -2
Diferencia máxima: 0

📋 DISTRIBUCIÓN DE DIFERENCIAS (rooms - bedrooms):
   Diferencia  -2:      1 casos (  0.0%)
   Diferencia   0: 32,837 casos (100.0%)

🔍 EJEMPLOS DE REGISTROS CON DIFERENCIAS:
        rooms  bedrooms  diferencia


In [11]:
# ===============================================================
# RECUPERAR DATOS DE ROOMS ANTES DE ELIMINAR COLUMNA
# ===============================================================

print("🔄 RECUPERACIÓN DE DATOS: ROOMS → BEDROOMS")
print("=" * 45)

if 'rooms' in df_clean.columns and 'bedrooms' in df_clean.columns:
    # PASO 1: Identificar casos donde rooms tiene valor pero bedrooms no
    print("🔍 PASO 1: IDENTIFICAR DATOS RECUPERABLES")
    print("-" * 40)
    
    # Casos donde rooms tiene valor y bedrooms está vacío
    mask_recuperable = df_clean['rooms'].notna() & df_clean['bedrooms'].isna()
    casos_recuperables = mask_recuperable.sum()
    
    print(f"📊 Registros con rooms pero sin bedrooms: {casos_recuperables:,}")
    
    if casos_recuperables > 0:
        # PASO 2: Recuperar los datos
        print(f"\n🔄 PASO 2: RECUPERAR DATOS")
        print("-" * 25)
        
        # Mostrar algunos ejemplos antes de la recuperación
        print("📋 Ejemplos antes de la recuperación:")
        ejemplos = df_clean[mask_recuperable][['rooms', 'bedrooms']].head()
        print(ejemplos)
        
        # Copiar valores de rooms a bedrooms donde bedrooms está vacío
        df_clean.loc[mask_recuperable, 'bedrooms'] = df_clean.loc[mask_recuperable, 'rooms']
        
        # Verificar la recuperación
        print(f"\n✅ Datos recuperados exitosamente: {casos_recuperables:,} registros")
        
        # Mostrar los mismos ejemplos después de la recuperación
        print("📋 Ejemplos después de la recuperación:")
        ejemplos_despues = df_clean[mask_recuperable][['rooms', 'bedrooms']].head()
        print(ejemplos_despues)
        
    else:
        print("✅ No hay datos para recuperar (todos los registros con rooms ya tienen bedrooms)")
    
    # PASO 3: Estadísticas finales antes de eliminar
    print(f"\n📊 ESTADÍSTICAS FINALES ANTES DE ELIMINAR ROOMS:")
    print("-" * 50)
    
    rooms_completos = df_clean['rooms'].notna().sum()
    bedrooms_completos = df_clean['bedrooms'].notna().sum()
    
    print(f"Rooms completos:    {rooms_completos:,}")
    print(f"Bedrooms completos: {bedrooms_completos:,}")
    print(f"Ganancia de datos:  {casos_recuperables:,}")
    
    # PASO 4: Eliminar la columna rooms
    print(f"\n🗑️ PASO 3: ELIMINAR COLUMNA REDUNDANTE")
    print("-" * 35)
    
    df_clean = df_clean.drop(columns=['rooms'])
    
    print(f"✅ Columna 'rooms' eliminada exitosamente")
    print(f"💾 Datos preservados en 'bedrooms'")
    
else:
    print("❌ Una o ambas columnas no encontradas")

print(f"\n✅ Dataset actualizado: {df_clean.shape[0]:,} registros, {df_clean.shape[1]} columnas")

🔄 RECUPERACIÓN DE DATOS: ROOMS → BEDROOMS
🔍 PASO 1: IDENTIFICAR DATOS RECUPERABLES
----------------------------------------
📊 Registros con rooms pero sin bedrooms: 0
✅ No hay datos para recuperar (todos los registros con rooms ya tienen bedrooms)

📊 ESTADÍSTICAS FINALES ANTES DE ELIMINAR ROOMS:
--------------------------------------------------
Rooms completos:    32,838
Bedrooms completos: 71,116
Ganancia de datos:  0

🗑️ PASO 3: ELIMINAR COLUMNA REDUNDANTE
-----------------------------------
✅ Columna 'rooms' eliminada exitosamente
💾 Datos preservados en 'bedrooms'

✅ Dataset actualizado: 277,616 registros, 21 columnas


In [12]:
# ===============================================================
# FILTRAR SOLO PROPIEDADES EN VENTA (ELIMINAR ARRIENDOS)
# ===============================================================

print("🏠 FILTRO: SOLO PROPIEDADES EN VENTA")
print("=" * 40)

# Mostrar distribución antes del filtro
antes = len(df_clean)
print(f"📊 Antes: {antes:,} propiedades")
print(f"Distribución: {df_clean['operation_type'].value_counts().to_dict()}")

# Filtrar solo propiedades en venta
df_clean = df_clean[df_clean['operation_type'] == 'Venta'].copy()

# Mostrar resultado
despues = len(df_clean) 
eliminados = antes - despues
print(f"\n✅ Después: {despues:,} propiedades")
print(f"🗑️ Eliminados: {eliminados:,} arriendos")
print(f"📈 Conservado: {(despues/antes*100):.1f}%")

🏠 FILTRO: SOLO PROPIEDADES EN VENTA
📊 Antes: 277,616 propiedades
Distribución: {'Venta': 140435, 'Arriendo': 137127, 'Arriendo temporal': 54}

✅ Después: 140,435 propiedades
🗑️ Eliminados: 137,181 arriendos
📈 Conservado: 50.6%


## 7️⃣ **Extracción por Text Mining** 💎

**Decisión:** Extraer información de superficie desde descripciones usando expresiones regulares

**Justificación:** Muchas propiedades tienen superficie en el campo `description` pero no en `surface_total`. Esta es una **innovación clave** que recupera datos valiosos perdidos

In [13]:
# ===============================================================
# EXTRACCIÓN TEXT MINING - FUNCIÓN OPTIMIZADA
# ===============================================================

print("⛏️ TEXT MINING - SUPERFICIE OPTIMIZADA")
print("-" * 40)

def extraer_superficie_optimizada(descripcion):
    """Función optimizada para extraer superficie con múltiples patrones"""
    if pd.isna(descripcion):
        return None
    
    desc_lower = str(descripcion).lower()
    
    # Lista de patrones ordenados por especificidad
    patrones = [
        # Patrones principales (más específicos)
        r'(\d+(?:[.,]\d+)?)\s*(?:m2|m²|metros\s*cuadrados)',
        r'(\d+(?:[.,]\d+)?)\s*(?:mts2|mt2|metros2)',
        r'(\d+(?:[.,]\d+)?)\s*(?:metros|mts|metro)\s*(?:cuadrados?|construidos?)',
        
        # Patrones con contexto (incluye variantes sin tilde y abreviaciones)
        r'(?:área|area)\s*(?:de\s*|total\s*|construida\s*)?(\d+(?:[.,]\d+)?)',
        r'superficie\s*(?:de\s*|total\s*)?(\d+(?:[.,]\d+)?)',
        r'construidos?\s*(\d+(?:[.,]\d+)?)',
        r'(\d+(?:[.,]\d+)?)\s*(?:metros\s*construidos?)',
        
        # Patrones adicionales comunes
        r'(\d+(?:[.,]\d+)?)\s*(?:mts?|mt)\s*(?:$|\s|[^a-z0-9])',  # Para "9mt", "120mts"
        r'(\d+(?:[.,]\d+)?)\s*m\s*(?:cuadrados?|construidos?)',
        
        # Patrones contextuales adicionales
        r'(?:tamaño|tamano)\s*(?:de\s*)?(\d+(?:[.,]\d+)?)',
        r'(\d+(?:[.,]\d+)?)\s*(?:de\s*área|de\s*area)',
    ]
    
    for patron in patrones:
        matches = re.findall(patron, desc_lower)
        if matches:
            superficie_str = matches[0].replace(',', '.')
            try:
                valor = float(superficie_str)
                if 15 <= valor <= 2000:  # Rango razonable
                    return valor
            except:
                continue
    return None

# Análisis inicial
sin_superficie = df_clean['surface_total'].isna().sum()
print(f"🔍 Registros sin superficie: {sin_superficie:,}")

# Aplicar extracción optimizada
mask_sin_superficie = df_clean['surface_total'].isna()

# Limpiar columna anterior si existe
if 'surface_extracted' in df_clean.columns:
    df_clean = df_clean.drop('surface_extracted', axis=1)

# Crear nueva columna con extracciones
extracciones = df_clean.loc[mask_sin_superficie, 'description'].apply(extraer_superficie_optimizada)
df_clean.loc[mask_sin_superficie, 'surface_extracted'] = extracciones

# Estadísticas finales
extraidas = df_clean['surface_extracted'].notna().sum()
print(f"✅ Superficies extraídas: {extraidas:,}")
print(f"📈 Tasa de recuperación: {(extraidas/sin_superficie*100):.1f}%")
print(f"💎 Innovación text mining completada")

⛏️ TEXT MINING - SUPERFICIE OPTIMIZADA
----------------------------------------
🔍 Registros sin superficie: 137,798
✅ Superficies extraídas: 45,755
📈 Tasa de recuperación: 33.2%
💎 Innovación text mining completada


In [14]:
# ===============================================================
# FUNCIONES DE EXTRACCIÓN PARA HABITACIONES Y BAÑOS
# ===============================================================

print("🔧 CREANDO FUNCIONES DE EXTRACCIÓN")
print("-" * 40)

def extraer_bedrooms(descripcion):
    """Extrae número de dormitorios/habitaciones desde descripción con patrones mejorados"""
    if pd.isna(descripcion):
        return None
    
    desc_lower = str(descripcion).lower()
    
    # Patrones mejorados (basados en celda 20)
    patrones = [
        # Variantes con errores tipográficos comunes
        r'(\d+)\s*(?:habitación|habitacion|habitaciom|avitacion|abitación)(?:es)?',
        r'(?:habitación|habitacion|habitaciom|avitacion|abitación)(?:es)?\s*(\d+)',
        
        # Sinónimos comunes
        r'(\d+)\s*(?:dormitorio|cuarto|alcoba|recamara|pieza)(?:s)?',
        r'(?:dormitorio|cuarto|alcoba|recamara|pieza)(?:s)?\s*(\d+)',
        
        # Abreviaciones (con y sin punto)
        r'(\d+)\s*hab\.?(?:s)?[^a-z]',
        r'(\d+)\s*dorm\.?(?:s)?[^a-z]',
        r'(\d+)\s*alcob\.?(?:s)?[^a-z]',
        
        # Expresiones numéricas en texto (convertir a números)
        r'(?:una|un)\s*(?:habitación|habitacion|dormitorio|cuarto|alcoba)',
        r'(?:dos)\s*(?:habitaciones|habitacion|dormitorios|cuartos|alcobas)',
        r'(?:tres)\s*(?:habitaciones|habitacion|dormitorios|cuartos|alcobas)',
        r'(?:cuatro)\s*(?:habitaciones|habitacion|dormitorios|cuartos|alcobas)',
        
        # Patrones contextuales
        r'(\d+)\s*(?:número\s*de\s*)?(?:habitaciones|dormitorios|cuartos)',
    ]
    
    # Diccionario para convertir texto a números
    texto_a_numero = {
        'una': 1, 'un': 1, 'dos': 2, 'tres': 3, 'cuatro': 4, 
        'cinco': 5, 'seis': 6, 'siete': 7, 'ocho': 8
    }
    
    for i, patron in enumerate(patrones):
        matches = re.findall(patron, desc_lower)
        if matches:
            try:
                # Para patrones de texto (una, dos, tres...)
                if i >= 7 and i <= 10:  # Patrones de texto
                    for palabra in texto_a_numero:
                        if palabra in desc_lower:
                            return texto_a_numero[palabra]
                else:
                    # Para patrones numéricos
                    valor = int(matches[0])
                    if 1 <= valor <= 10:  # Rango razonable
                        return valor
            except:
                continue
    return None

def extraer_bathrooms(descripcion):
    """Extrae número de baños desde descripción con patrones mejorados"""
    if pd.isna(descripcion):
        return None
    
    desc_lower = str(descripcion).lower()
    
    # Patrones mejorados (basados en celda 20)
    patrones = [
        # Variantes con errores tipográficos
        r'(\d+)\s*(?:baño|bano|banio|banyo|bañio)(?:s)?',
        r'(?:baño|bano|banio|banyo|bañio)(?:s)?\s*(\d+)',
        
        # Sinónimos y variantes
        r'(\d+)\s*(?:bathroom|wc|w\.c\.|sanitario|aseo|toilet|toillet)(?:s)?',
        r'(?:bathroom|wc|w\.c\.|sanitario|aseo|toilet|toillet)(?:s)?\s*(\d+)',
        
        # Abreviaciones
        r'(\d+)\s*bath(?:s)?[^a-z]',
        r'(\d+)\s*b\.(?:s)?[^a-z]',
        
        # Tipos específicos de baños
        r'(\d+)\s*(?:baño\s*completo|baño\s*social|baño\s*de\s*visitas)',
        r'(?:baño\s*completo|baño\s*social|baño\s*de\s*visitas)(?:s)?\s*(\d+)',
        
        # Expresiones numéricas en texto
        r'(?:un)\s*(?:baño|bano|bathroom|sanitario)',
        r'(?:dos)\s*(?:baños|banos|bathrooms|sanitarios)',
        r'(?:tres)\s*(?:baños|banos|bathrooms|sanitarios)',
        
        # Medio baño (manejado especialmente como 0.5)
        r'(?:medio\s*baño|medio\s*bano|baño\s*auxiliar)',
    ]
    
    # Diccionario para convertir texto a números
    texto_a_numero = {
        'un': 1, 'dos': 2, 'tres': 3, 'cuatro': 4, 
        'cinco': 5, 'seis': 6
    }
    
    # Verificar medio baño primero (retorna 0.5)
    if re.search(r'(?:medio\s*baño|medio\s*bano|baño\s*auxiliar)', desc_lower):
        return 0.5
    
    for i, patron in enumerate(patrones):
        matches = re.findall(patron, desc_lower)
        if matches:
            try:
                # Para patrones de texto (un, dos, tres...)
                if i >= 8 and i <= 10:  # Patrones de texto
                    for palabra in texto_a_numero:
                        if palabra in desc_lower and 'baño' in desc_lower:
                            return texto_a_numero[palabra]
                else:
                    # Para patrones numéricos
                    valor = int(matches[0])
                    if 1 <= valor <= 8:  # Rango razonable
                        return valor
            except:
                continue
    return None

print("✅ Funciones de extracción creadas")

# Probar funciones en muestra pequeña
print(f"\n🧪 PRUEBA DE FUNCIONES:")
print("-" * 25)



🔧 CREANDO FUNCIONES DE EXTRACCIÓN
----------------------------------------
✅ Funciones de extracción creadas

🧪 PRUEBA DE FUNCIONES:
-------------------------


In [15]:
# ===============================================================
# APLICAR EXTRACCIÓN AL DATASET COMPLETO
# ===============================================================

print("🚀 APLICACIÓN AL DATASET COMPLETO")
print("=" * 40)

# Aplicar solo a registros que no tienen la información
variables_extraer = {
    'bedrooms': extraer_bedrooms,
    'bathrooms': extraer_bathrooms
}

for variable, funcion in variables_extraer.items():
    print(f"\n🔄 Procesando {variable}...")
    
    # Verificar si la columna existe
    if variable in df_clean.columns:
        faltantes_inicial = df_clean[variable].isna().sum()
        mask_faltantes = df_clean[variable].isna()
    else:
        print(f"   Creando nueva columna '{variable}'")
        df_clean[variable] = np.nan
        faltantes_inicial = len(df_clean)
        mask_faltantes = df_clean[variable].isna()
    
    print(f"   Registros sin {variable}: {faltantes_inicial:,}")
    
    # Aplicar extracción
    extracciones = df_clean.loc[mask_faltantes, 'description'].apply(funcion)
    df_clean.loc[mask_faltantes, f'{variable}_extracted'] = extracciones
    
    # Estadísticas - CORREGIDO: calcular extraídos específicos para esta variable
    extraidos = (df_clean.loc[mask_faltantes, f'{variable}_extracted'].notna()).sum()
    tasa_recuperacion = (extraidos / faltantes_inicial * 100) if faltantes_inicial > 0 else 0
    
    print(f"   ✅ Extraídos: {extraidos:,}")
    print(f"   📈 Tasa recuperación: {tasa_recuperacion:.1f}%")

print(f"\n💎 EXTRACCIÓN ADICIONAL COMPLETADA")
print("=" * 40)

# Resumen final de extracciones
print(f"📊 RESUMEN DE TODAS LAS EXTRACCIONES:")
print("-" * 35)
print(f"   Surface: {df_clean['surface_extracted'].notna().sum():,} extraídas")
print(f"   Bedrooms: {df_clean['bedrooms_extracted'].notna().sum():,} extraídas")
print(f"   Bathrooms: {df_clean['bathrooms_extracted'].notna().sum():,} extraídas")

total_extracciones = (
    df_clean['surface_extracted'].notna().sum() +
    df_clean['bedrooms_extracted'].notna().sum() +
    df_clean['bathrooms_extracted'].notna().sum()
)

print(f"\n🎯 TOTAL DATOS RECUPERADOS: {total_extracciones:,}")
print(f"🏆 Text Mining: INNOVACIÓN COMPLETA")

🚀 APLICACIÓN AL DATASET COMPLETO

🔄 Procesando bedrooms...
   Registros sin bedrooms: 102,070
   ✅ Extraídos: 81,447
   📈 Tasa recuperación: 79.8%

🔄 Procesando bathrooms...
   Registros sin bathrooms: 18,727
   ✅ Extraídos: 10,795
   📈 Tasa recuperación: 57.6%

💎 EXTRACCIÓN ADICIONAL COMPLETADA
📊 RESUMEN DE TODAS LAS EXTRACCIONES:
-----------------------------------
   Surface: 45,755 extraídas
   Bedrooms: 81,447 extraídas
   Bathrooms: 10,795 extraídas

🎯 TOTAL DATOS RECUPERADOS: 137,997
🏆 Text Mining: INNOVACIÓN COMPLETA


## 8️⃣ **Integración de Datos Extraídos**

**Decisión:** Consolidar datos originales con extracciones de text mining

**Justificación:** Crear variables finales que combinen datos originales con extracciones para maximizar completitud sin perder información original

In [16]:
# ===============================================================
# INTEGRACIÓN DE DATOS ORIGINALES CON EXTRACCIONES
# ===============================================================

print("🔗 INTEGRACIÓN DE DATOS")
print("=" * 30)

# Estrategia: Priorizar datos originales, completar con extracciones
variables_integrar = {
    'surface_total': 'surface_extracted',
    'bedrooms': 'bedrooms_extracted', 
    'bathrooms': 'bathrooms_extracted'
}

for original, extraida in variables_integrar.items():
    if extraida in df_clean.columns:
        # Estado antes de integración
        antes_faltantes = df_clean[original].isna().sum()
        
        # Integrar: usar original si existe, sino usar extraída
        df_clean[f'{original}_final'] = df_clean[original].fillna(df_clean[extraida])
        
        # Estado después de integración
        despues_faltantes = df_clean[f'{original}_final'].isna().sum()
        completados = antes_faltantes - despues_faltantes
        
        print(f"\n📊 {original.upper()}:")
        print(f"   Antes: {antes_faltantes:,} faltantes")
        print(f"   Después: {despues_faltantes:,} faltantes")
        print(f"   ✅ Completados: {completados:,}")
        if antes_faltantes > 0:
            print(f"   📈 Mejora: {(completados/antes_faltantes*100):.1f}%")

print(f"\n🎯 RESUMEN DE INTEGRACIÓN:")
print("-" * 30)

# Calcular total de completaciones
total_completaciones = 0
for original, extraida in variables_integrar.items():
    if extraida in df_clean.columns:
        antes = df_clean[original].isna().sum()
        despues = df_clean[f'{original}_final'].isna().sum()
        total_completaciones += (antes - despues)

print(f"📈 Total valores completados: {total_completaciones:,}")
print(f"💎 Integración exitosa completada")

print(f"Eliminar columnas temporales")

df_clean.drop(['surface_extracted', 'bedrooms_extracted', 'bathrooms_extracted'], axis=1, inplace=True)


🔗 INTEGRACIÓN DE DATOS

📊 SURFACE_TOTAL:
   Antes: 137,798 faltantes
   Después: 92,043 faltantes
   ✅ Completados: 45,755
   📈 Mejora: 33.2%

📊 BEDROOMS:
   Antes: 102,070 faltantes
   Después: 20,623 faltantes
   ✅ Completados: 81,447
   📈 Mejora: 79.8%

📊 BATHROOMS:
   Antes: 18,727 faltantes
   Después: 7,932 faltantes
   ✅ Completados: 10,795
   📈 Mejora: 57.6%

🎯 RESUMEN DE INTEGRACIÓN:
------------------------------
📈 Total valores completados: 137,997
💎 Integración exitosa completada
Eliminar columnas temporales


In [18]:
# ===============================================================
# ANÁLISIS DE VALORES FALTANTES RESTANTES
# ===============================================================

print("🔍 ANÁLISIS DE VALORES FALTANTES RESTANTES")
print("=" * 50)

# Evaluar estado actual después de text mining e integración
variables_criticas = ['lat', 'lon', 'l4', 'surface_total_final', 'bedrooms_final', 'bathrooms_final']

print("📊 ESTADO ACTUAL DE VARIABLES CRÍTICAS:")
print("-" * 45)

for var in variables_criticas:
    if var in df_clean.columns:
        faltantes = df_clean[var].isna().sum()
        total = len(df_clean)
        pct = (faltantes / total) * 100
        print(f"   {var:<20}: {faltantes:>8,} faltantes ({pct:>5.1f}%)")

# Identificar variables para imputación simple
print(f"\n🎯 ESTRATEGIAS DE IMPUTACIÓN:")
print("-" * 35)

# 1. Coordenadas faltantes - imputar por centroide de barrio
coords_faltantes = df_clean[['lat', 'lon']].isna().any(axis=1).sum()
barrios_disponibles = df_clean['l4'].notna().sum()
print(f"   Coordenadas: {coords_faltantes:,} faltantes")
print(f"   Barrios disponibles: {barrios_disponibles:,}")

# 2. Barrios faltantes - estrategia compleja (mantener para análisis)
barrios_faltantes = df_clean['l4'].isna().sum()
print(f"   Barrios: {barrios_faltantes:,} faltantes")

print(f"\n✅ ANÁLISIS COMPLETADO - LISTO PARA IMPUTACIÓN")

🔍 ANÁLISIS DE VALORES FALTANTES RESTANTES
📊 ESTADO ACTUAL DE VARIABLES CRÍTICAS:
---------------------------------------------
   lat                 :   78,250 faltantes ( 55.7%)
   lon                 :   78,250 faltantes ( 55.7%)
   l4                  :  107,750 faltantes ( 76.7%)
   surface_total_final :   92,043 faltantes ( 65.5%)
   bedrooms_final      :   20,623 faltantes ( 14.7%)
   bathrooms_final     :    7,932 faltantes (  5.6%)

🎯 ESTRATEGIAS DE IMPUTACIÓN:
-----------------------------------
   Coordenadas: 78,250 faltantes
   Barrios disponibles: 32,685
   Barrios: 107,750 faltantes

✅ ANÁLISIS COMPLETADO - LISTO PARA IMPUTACIÓN


## 🔟 **Extracción de Ubicación por Text Mining** 🌍

**Objetivo:** Extraer información de ubicación (barrios, sectores, zonas) desde las descripciones

**Justificación:** Con 211,876 registros sin barrio (l4) pero con descripciones disponibles, esta estrategia puede recuperar información geográfica valiosa que está "escondida" en el texto libre.

**Estrategia:** Usar patrones específicos para identificar menciones de barrios, sectores y zonas conocidas de Antioquia

In [19]:
# ===============================================================
# ANÁLISIS DE DESCRIPCIONES PARA EXTRACCIÓN DE UBICACIÓN
# ===============================================================

print("🌍 EXTRACCIÓN DE UBICACIÓN POR TEXT MINING")
print("=" * 50)

# 1. Estado actual de ubicaciones
print("📊 ESTADO ACTUAL DE UBICACIÓN:")
print("-" * 35)

total_registros = len(df_clean)
con_barrio = df_clean['l4'].notna().sum()
sin_barrio = df_clean['l4'].isna().sum()
con_descripcion = df_clean['description'].notna().sum()

print(f"Total registros: {total_registros:,}")
print(f"Con barrio (l4): {con_barrio:,} ({(con_barrio/total_registros*100):.1f}%)")
print(f"Sin barrio (l4): {sin_barrio:,} ({(sin_barrio/total_registros*100):.1f}%)")
print(f"Con descripción: {con_descripcion:,} ({(con_descripcion/total_registros*100):.1f}%)")

# 2. Candidatos para extracción: sin barrio PERO con descripción
candidatos_ubicacion = (df_clean['l4'].isna() & df_clean['description'].notna()).sum()
print(f"\n🎯 Candidatos para extracción: {candidatos_ubicacion:,}")
print(f"   (Sin barrio PERO con descripción)")

# 3. Obtener barrios conocidos para crear patrones
print(f"\n📋 BARRIOS CONOCIDOS EN EL DATASET:")
print("-" * 35)

barrios_conocidos = df_clean['l4'].dropna().unique()
print(f"Total barrios únicos: {len(barrios_conocidos):,}")

# Mostrar los más comunes
top_barrios = df_clean['l4'].value_counts().head(20)
print(f"\nTop 20 barrios más comunes:")
for i, (barrio, count) in enumerate(top_barrios.items(), 1):
    print(f"  {i:2d}. {barrio:<30}: {count:>4,} propiedades")

🌍 EXTRACCIÓN DE UBICACIÓN POR TEXT MINING
📊 ESTADO ACTUAL DE UBICACIÓN:
-----------------------------------
Total registros: 140,435
Con barrio (l4): 32,685 (23.3%)
Sin barrio (l4): 107,750 (76.7%)
Con descripción: 140,222 (99.8%)

🎯 Candidatos para extracción: 107,607
   (Sin barrio PERO con descripción)

📋 BARRIOS CONOCIDOS EN EL DATASET:
-----------------------------------
Total barrios únicos: 21

Top 20 barrios más comunes:
   1. El Poblado                    : 11,194 propiedades
   2. Laureles                      : 5,162 propiedades
   3. Belén                         : 4,298 propiedades
   4. La América                    : 2,513 propiedades
   5. Robledo                       : 1,766 propiedades
   6. Candelaria                    : 1,611 propiedades
   7. Buenos Aires                  : 1,321 propiedades
   8. Castilla                      :  728 propiedades
   9. San Cristóbal                 :  539 propiedades
  10. Guayabal                      :  533 propiedades
  11. Alt

In [20]:
# ===============================================================
# FUNCIONES UNIFICADAS DE EXTRACCIÓN DE UBICACIÓN
# ===============================================================

print("🏗️ CREANDO FUNCIONES UNIFICADAS DE EXTRACCIÓN DE UBICACIÓN")
print("=" * 60)

import re

# ===============================================================
# DATOS DE REFERENCIA: CIUDADES Y BARRIOS
# ===============================================================

# Lista completa de ciudades de Antioquia
CIUDADES_ANTIOQUIA = [
    # Ciudades principales del área metropolitana (prioridad)
    'medellín', 'medellin', 'envigado', 'itagüí', 'itagui', 'sabaneta', 
    'bello', 'copacabana', 'la estrella', 'estrella', 'caldas', 
    'girardota', 'barbosa', 'rionegro',
    
    # Todas las ciudades de Antioquia (alfabéticamente)
    'abejorral', 'abriaquí', 'alejandría', 'amagá', 'amalfi', 'andes', 
    'angelópolis', 'angostura', 'anorí', 'anzá', 'apartadó', 'arboletes', 
    'argelia', 'armenia', 'betania', 'betulia', 'briceño', 'buriticá', 
    'cáceres', 'caicedo', 'campamento', 'cañasgordas', 'caracolí', 
    'caramanta', 'carepa', 'carolina del príncipe', 'caucasia', 
    'chigorodó', 'cisneros', 'ciudad bolívar', 'cocorná', 'concepción', 
    'concordia', 'dabeiba', 'don matías', 'ebéjico', 'el bagre', 
    'entrerríos', 'fredonia', 'frontino', 'giraldo', 'gómez plata', 
    'granada', 'guadalupe', 'guarne', 'guatapé', 'heliconia', 'hispania', 
    'jardín', 'jericó', 'la ceja', 'la pintada', 'la unión', 'liborina', 
    'maceo', 'marinilla', 'montebello', 'murindó', 'mutatá', 'nariño', 
    'nechí', 'necoclí', 'olaya', 'peñol', 'peque', 'pueblorrico', 
    'puerto berrío', 'puerto nare', 'puerto triunfo', 'remedios', 
    'retiro', 'salgar', 'san andrés de cuerquia', 'san carlos', 
    'san francisco', 'san jerónimo', 'san josé de la montaña', 
    'san luis', 'san pedro de los milagros', 'san pedro de urabá', 
    'san rafael', 'san roque', 'san vicente', 'santa bárbara', 
    'santa rosa de osos', 'santo domingo', 'segovia', 'sonsón', 
    'sopetrán', 'támesis', 'tarazá', 'tarso', 'titiribí', 'toledo', 
    'turbo', 'uramita', 'urrao', 'valdivia', 'valparaíso', 'vegachí', 
    'venecia', 'vigía del fuerte', 'yalí', 'yarumal', 'yolombó', 'yondó'
]

# Lista de barrios de Antioquia organizados por municipio/comuna
BARRIOS_ANTIOQUIA = [
    # ITAGÜÍ
    'calatrava', 'santa maría', 'ditaires', 'san gabriel',
    
    # ENVIGADO  
    'la frontera', 'jardines', 'el dorado', 'las palmas',
    
    # SABANETA
    'la doctora', 'holanda', 'mayorca',
    
    # LA ESTRELLA
    'el pedrero', 'san andrés', 'bellavista',
    
    # CALDAS
    'primavera', 'la tablaza', 'los lagos',
    
    # BELLO
    'niquía', 'cabañas', 'zamora', 'parís', 'fontidueño',
    
    # COPACABANA
    'la asunción', 'la misericordia', 'machado',
    
    # GIRARDOTA
    'san esteban', 'aurelio mejía',
    
    # BARBOSA
    'el porvenir', 'la estación',
    
    # MEDELLÍN - COMUNA 1 (POPULAR)
    'santo domingo savio', 'popular', 'granizal',
    
    # COMUNA 2 (SANTA CRUZ)
    'la isla', 'la rosa', 'santa cruz',
    
    # COMUNA 3 (MANRIQUE) 
    'manrique central', 'manrique', 'versalles', 'el raizal',
    
    # COMUNA 4 (ARANJUEZ)
    'moravia', 'aranjuez', 'san pedro',
    
    # COMUNA 5 (CASTILLA)
    'castilla', 'tricentenario', 'alfonso lópez',
    
    # COMUNA 6 (DOCE DE OCTUBRE)
    'pedregal', 'doce de octubre', 'kennedy',
    
    # COMUNA 7 (ROBLEDO)
    'robledo', 'el volador', 'la pilarica', 'pajarito',
    
    # COMUNA 8 (VILLA HERMOSA)
    'villa hermosa', 'la milagrosa', 'golondrinas',
    
    # COMUNA 9 (BUENOS AIRES)
    'buenos aires', 'la asomadera', 'los cerros',
    
    # COMUNA 10 (LA CANDELARIA - CENTRO)
    'la candelaria', 'candelaria', 'prado', 'san benito', 'guayaquil',
    
    # COMUNA 11 (LAURELES - ESTADIO)
    'laureles', 'estadio', 'los colores', 'san joaquín',
    
    # COMUNA 12 (LA AMÉRICA)
    'la américa', 'américa', 'santa lucía', 'la floresta', 'floresta', 'calasanz',
    
    # COMUNA 13 (SAN JAVIER)
    'san javier', 'el salado', 'blanquizal', 'las independencias',
    
    # COMUNA 14 (EL POBLADO) 
    'el poblado', 'poblado', 'astorga', 'manila', 'provenza', 'castropol',
    
    # COMUNA 15 (GUAYABAL)
    'guayabal', 'cristo rey', 'trinidad', 'campo amor',
    
    # COMUNA 16 (BELÉN)
    'belén', 'belen', 'loma de los bernal', 'fátima', 'los alpes',
    
    # CORREGIMIENTOS MEDELLÍN (RURALES)
    'san sebastián de palmitas', 'palmitas', 'san cristóbal', 'san cristobal', 
    'altavista', 'san antonio de prado', 'santa elena'
]

# ===============================================================
# FUNCIONES DE MAPEO A NOMBRES ESTÁNDAR
# ===============================================================

def mapear_ciudad_estandar(ciudad_encontrada):
    """Mapea ciudades encontradas a nombres estándar del dataset"""
    ciudad_lower = ciudad_encontrada.lower()
    
    # Diccionario de mapeos
    mapeos_ciudades = {
        'medellin': 'Medellín', 'medellín': 'Medellín',
        'itagui': 'Itagüí', 'itagüí': 'Itagüí',
        'estrella': 'La Estrella', 'la estrella': 'La Estrella',
        'carolina del príncipe': 'Carolina del Príncipe',
        'ciudad bolívar': 'Ciudad Bolívar',
        'don matías': 'Don Matías',
        'gómez plata': 'Gómez Plata',
        'la ceja': 'La Ceja',
        'la pintada': 'La Pintada',
        'la unión': 'La Unión',
        'puerto berrío': 'Puerto Berrío',
        'puerto nare': 'Puerto Nare',
        'puerto triunfo': 'Puerto Triunfo',
        'san andrés de cuerquia': 'San Andrés de Cuerquia',
        'san carlos': 'San Carlos',
        'san francisco': 'San Francisco',
        'san jerónimo': 'San Jerónimo',
        'san josé de la montaña': 'San José de la Montaña',
        'san luis': 'San Luis',
        'san pedro de los milagros': 'San Pedro de los Milagros',
        'san pedro de urabá': 'San Pedro de Urabá',
        'san rafael': 'San Rafael',
        'san roque': 'San Roque',
        'san vicente': 'San Vicente',
        'santa bárbara': 'Santa Bárbara',
        'santa rosa de osos': 'Santa Rosa de Osos',
        'santo domingo': 'Santo Domingo',
        'vigía del fuerte': 'Vigía del Fuerte'
    }
    
    return mapeos_ciudades.get(ciudad_lower, ciudad_encontrada.title())

def mapear_barrio_estandar(barrio_encontrado):
    """Mapea barrios encontrados a nombres estándar del dataset"""
    barrio_lower = barrio_encontrado.lower()
    
    # Diccionario de mapeos
    mapeos_barrios = {
        'el poblado': 'El Poblado', 'poblado': 'El Poblado',
        'belen': 'Belén', 'belén': 'Belén',
        'la américa': 'La América', 'américa': 'La América',
        'la floresta': 'La Floresta', 'floresta': 'La Floresta',
        'la candelaria': 'La Candelaria', 'candelaria': 'La Candelaria',
        'san cristóbal': 'San Cristóbal', 'san cristobal': 'San Cristóbal',
        'santo domingo savio': 'Santo Domingo Savio',
        'manrique central': 'Manrique Central',
        'doce de octubre': 'Doce de Octubre',
        'villa hermosa': 'Villa Hermosa',
        'buenos aires': 'Buenos Aires',
        'los colores': 'Los Colores',
        'san joaquín': 'San Joaquín',
        'santa lucía': 'Santa Lucía',
        'san javier': 'San Javier',
        'el salado': 'El Salado',
        'las independencias': 'Las Independencias',
        'cristo rey': 'Cristo Rey',
        'campo amor': 'Campo Amor',
        'loma de los bernal': 'Loma de Los Bernal',
        'los alpes': 'Los Alpes',
        'san sebastián de palmitas': 'San Sebastián de Palmitas',
        'san antonio de prado': 'San Antonio de Prado',
        'santa maría': 'Santa María',
        'san gabriel': 'San Gabriel',
        'la frontera': 'La Frontera',
        'el dorado': 'El Dorado',
        'las palmas': 'Las Palmas',
        'la doctora': 'La Doctora',
        'el pedrero': 'El Pedrero',
        'san andrés': 'San Andrés',
        'la tablaza': 'La Tablaza',
        'los lagos': 'Los Lagos',
        'la asunción': 'La Asunción',
        'la misericordia': 'La Misericordia',
        'san esteban': 'San Esteban',
        'aurelio mejía': 'Aurelio Mejía',
        'el porvenir': 'El Porvenir',
        'la estación': 'La Estación',
        'la isla': 'La Isla',
        'la rosa': 'La Rosa',
        'el raizal': 'El Raizal',
        'san pedro': 'San Pedro',
        'alfonso lópez': 'Alfonso López',
        'el volador': 'El Volador',
        'la pilarica': 'La Pilarica',
        'la milagrosa': 'La Milagrosa',
        'la asomadera': 'La Asomadera',
        'los cerros': 'Los Cerros',
        'san benito': 'San Benito'
    }
    
    return mapeos_barrios.get(barrio_lower, barrio_encontrado.title())

# ===============================================================
# FUNCIONES UNIFICADAS DE EXTRACCIÓN
# ===============================================================

def extraer_ciudad(texto):
    """
    Extrae ciudad desde cualquier campo de texto (description o title)
    
    Args:
        texto (str): Texto donde buscar la ciudad
        
    Returns:
        str or None: Ciudad encontrada en formato estándar o None
    """
    if pd.isna(texto):
        return None
    
    texto_lower = str(texto).lower()
    
    # Buscar cada ciudad con patrón de palabra completa
    for ciudad in CIUDADES_ANTIOQUIA:
        patron = rf'\b{re.escape(ciudad)}\b'
        if re.search(patron, texto_lower):
            return mapear_ciudad_estandar(ciudad)
    
    return None

def extraer_barrio(texto):
    """
    Extrae barrio desde cualquier campo de texto (description o title)
    
    Args:
        texto (str): Texto donde buscar el barrio
        
    Returns:
        str or None: Barrio encontrado en formato estándar o None
    """
    if pd.isna(texto):
        return None
    
    texto_lower = str(texto).lower()
    
    # Buscar cada barrio con patrón de palabra completa
    for barrio in BARRIOS_ANTIOQUIA:
        patron = rf'\b{re.escape(barrio)}\b'
        if re.search(patron, texto_lower):
            return mapear_barrio_estandar(barrio)
    
    return None

print("✅ Funciones unificadas de extracción creadas:")
print("   📍 extraer_ciudad(texto) - Funciona con description o title")
print("   📍 extraer_barrio(texto) - Funciona con description o title")

# ===============================================================
# APLICAR EXTRACCIONES AL DATASET COMPLETO
# ===============================================================

print(f"\n🚀 APLICANDO EXTRACCIONES AL DATASET COMPLETO")
print("=" * 55)

# Extracción desde descriptions
print(f"\n🏙️ EXTRAYENDO DESDE DESCRIPTIONS...")
df_clean['ciudad_desc'] = df_clean['description'].apply(extraer_ciudad)
df_clean['barrio_desc'] = df_clean['description'].apply(extraer_barrio)

# Extracción desde titles
print(f"🏘️ EXTRAYENDO DESDE TITLES...")
df_clean['ciudad_title'] = df_clean['title'].apply(extraer_ciudad)
df_clean['barrio_title'] = df_clean['title'].apply(extraer_barrio)

# ===============================================================
# CONSOLIDACIÓN FINAL EN COLUMNAS l3_final Y l4_final
# ===============================================================

print(f"\n🔗 CONSOLIDANDO EN COLUMNAS FINALES")
print("=" * 40)

# PASO 1: Crear l3_final (prioridad: l3 original → description → title)
df_clean['l3_final'] = df_clean['l3'].fillna(
    df_clean['ciudad_desc']
).fillna(
    df_clean['ciudad_title']
)

# PASO 2: Crear l4_final (prioridad: l4 original → description → title)  
df_clean['l4_final'] = df_clean['l4'].fillna(
    df_clean['barrio_desc']
).fillna(
    df_clean['barrio_title']
)

# PASO 3: Limpiar columnas temporales
df_clean.drop(['ciudad_desc', 'barrio_desc', 'ciudad_title', 'barrio_title'], axis=1, inplace=True)

print(f"✅ Consolidación completada:")
print(f"   📍 l3_final: Datos originales + extraídos (description + title)")
print(f"   📍 l4_final: Datos originales + extraídos (description + title)")

# ===============================================================
# REPORTE DE RESULTADOS
# ===============================================================

print(f"\n📊 REPORTE DE COBERTURA FINAL:")
print("=" * 40)

total_registros = len(df_clean)
ciudades_final = df_clean['l3_final'].notna().sum()
barrios_final = df_clean['l4_final'].notna().sum()
ciudades_original = df_clean['l3'].notna().sum()
barrios_original = df_clean['l4'].notna().sum()

print(f"🏙️ Ciudades:")
print(f"   Original (l3):    {ciudades_original:,} ({ciudades_original/total_registros*100:.1f}%)")
print(f"   Final (l3_final): {ciudades_final:,} ({ciudades_final/total_registros*100:.1f}%)")
print(f"   🎯 Ganancia:      +{ciudades_final - ciudades_original:,}")

print(f"\n🏘️ Barrios:")
print(f"   Original (l4):    {barrios_original:,} ({barrios_original/total_registros*100:.1f}%)")
print(f"   Final (l4_final): {barrios_final:,} ({barrios_final/total_registros*100:.1f}%)")
print(f"   🎯 Ganancia:      +{barrios_final - barrios_original:,}")



🏗️ CREANDO FUNCIONES UNIFICADAS DE EXTRACCIÓN DE UBICACIÓN
✅ Funciones unificadas de extracción creadas:
   📍 extraer_ciudad(texto) - Funciona con description o title
   📍 extraer_barrio(texto) - Funciona con description o title

🚀 APLICANDO EXTRACCIONES AL DATASET COMPLETO

🏙️ EXTRAYENDO DESDE DESCRIPTIONS...
🏘️ EXTRAYENDO DESDE TITLES...

🔗 CONSOLIDANDO EN COLUMNAS FINALES
✅ Consolidación completada:
   📍 l3_final: Datos originales + extraídos (description + title)
   📍 l4_final: Datos originales + extraídos (description + title)

📊 REPORTE DE COBERTURA FINAL:
🏙️ Ciudades:
   Original (l3):    137,499 (97.9%)
   Final (l3_final): 139,853 (99.6%)
   🎯 Ganancia:      +2,354

🏘️ Barrios:
   Original (l4):    32,685 (23.3%)
   Final (l4_final): 57,533 (41.0%)
   🎯 Ganancia:      +24,848


In [22]:
# 1. Información básica
print(f"📊 Dimensiones: {df_clean.shape}")
print(f"📊 Tipos de datos:")
print(df_clean.dtypes.value_counts())

df_clean.columns
# 2. Guardar dataset limpio y enriquecido
df_final = df_clean[['ad_type', 'start_date', 'end_date', 'created_on', 'lat', 'lon','price','title', 'description', 'property_type', 'operation_type','surface_total_final', 'bedrooms_final', 'bathrooms_final', 'l3_final', 'l4_final']]
df_final.to_csv('../data/properties_gold.csv', index=False)



📊 Dimensiones: (140435, 26)
📊 Tipos de datos:
object     16
float64    10
Name: count, dtype: int64
